In [1]:
import pandas as pd
import math
from helper.map import mapper
from helper.haversine import haversine
from helper.load_coordinates import load_coordinates
from helper.calculate_shortest_path import dijkstra
from helper.create_graph_from_csv import create_graph_from_csv
#from helper.plot_map import plot_map
map_obj = mapper(True)
map_obj


Total locations plotted : 85


In [2]:

# Load coordinates
df = pd.read_csv('../assets/location_coordinates_final.csv')  # Assumes columns: name, Latitude, Longitude

# Initialize matrix
stops = df['name'].tolist()
matrix_data = []

# Calculate pairwise distances
for index1, row1 in df.iterrows():
    row_distances = {'name': row1['name']}
    for index2, row2 in df.iterrows():
        dist = 0 if index1 == index2 else haversine(
            row1['Latitude'], row1['Longitude'],
            row2['Latitude'], row2['Longitude']
        )
        row_distances[row2['name']] = round(dist, 3)
    matrix_data.append(row_distances)

# Convert to DataFrame and save
dist_matrix_df = pd.DataFrame(matrix_data)
dist_matrix_df.to_csv('../assets/distances_log.csv')

print("✅ distances_matrix.csv created with", len(stops), "x", len(stops), "matrix.")


✅ distances_matrix.csv created with 86 x 86 matrix.


In [3]:
dist = pd.read_csv("../assets/distances_log.csv")

In [4]:
dist.head()

,Unnamed: 0,name,clock tower,darshanlal chowk,saharanpur chowk,isbt dehradun,graphic era hill university,asley hall,matawala bagh,daudwala,...,panditwari,vasant vihar,selaqui chowk,baliwala chowk,bahuwala raja road,raja road,bansiwala,manduwala,prem nagar,balliwala chowk
0,0,clock tower,0.000,0.212,1.457,5.840,6.965,0.567,2.103,7.497,...,5.200,4.073,25.993,0.212,1.148,0.952,14.117,11.985,7.821,0.212
1,1,darshanlal chowk,0.212,0.000,1.419,5.746,6.835,0.676,2.067,7.310,...,5.289,4.142,26.114,0.000,0.946,0.843,14.212,12.157,7.909,0.000
2,2,saharanpur chowk,1.457,1.419,0.000,4.428,5.665,2.021,0.648,6.607,...,4.159,2.944,25.067,1.419,1.316,0.679,13.054,11.534,6.742,1.419
3,3,isbt dehradun,5.840,5.746,4.428,0.000,1.698,6.405,3.838,4.339,...,4.788,4.110,23.798,5.746,5.122,4.904,11.646,12.500,6.154,5.746
4,4,graphic era hill university,6.965,6.835,5.665,1.698,0.000,7.511,5.154,3.032,...,6.484,5.789,24.897,6.835,6.065,6.013,12.843,14.134,7.707,6.835


In [5]:
# Build the graph
file_path = '../assets/distances_log.csv'
graph_obj = create_graph_from_csv(file_path)

# Display it (optional)
graph_obj.display_graph()


Missing value counts *before* fix:
Unnamed: 0          0
name                0
clock tower         1
darshanlal chowk    1
saharanpur chowk    1
                   ..
raja road           1
bansiwala           1
manduwala           1
prem nagar          1
balliwala chowk     1
Length: 88, dtype: int64
Missing value counts *after* fix:
Unnamed: 0          0
name                0
clock tower         0
darshanlal chowk    0
saharanpur chowk    0
                   ..
raja road           0
bansiwala           0
manduwala           0
prem nagar          0
balliwala chowk     0
Length: 88, dtype: int64
clock tower->darshanlal chowk (0.212),saharanpur chowk (1.457),isbt dehradun (5.84),graphic era hill university (6.965),asley hall (0.567),matawala bagh (2.103),daudwala (7.497),mathurawala (4.664),vishnupuram (4.606),bangali kothi (1.481),kargi chowk (4.353),ranipokhri (22.858),doiwala (18.273),lachiwala (16.683),kuanwala (1.006),harawala (5.875),miawala (5.045),mokampur (6.646),jogiwala (4.83

In [8]:
# Define start and end points
start_point = "clock tower"
end_point = "asley hall"

# Calculate shortest path
path, distance = dijkstra(graph_obj, start_point, end_point)

# Display results
if path is not None:
    print("📍 Path:", " -> ".join(path))
    print("📏 Distance:", round(distance, 2), "km")
else:
    print("No valid path exists between", start_point, "and", end_point)


📍 Path: clock tower -> asley hall
📏 Distance: 0.57 km


In [17]:
import folium
import pandas as pd

from helper.load_coordinates import load_coordinates
from helper.create_graph_from_csv import create_graph_from_csv
from helper.calculate_shortest_path import dijkstra

# Load coordinates and build coordinate dictionary
coord_dict, coords = load_coordinates("../assets/location_coordinates_final.csv")

# Create the graph from distance CSV
graph_obj = create_graph_from_csv("../assets/distances_log.csv")
graph_dict = {node: dict(graph_obj.graph[node]) for node in graph_obj.graph}


# Get Dijkstra path and distance
path, total_distance = dijkstra(graph_obj, start_point, end_point)

# Center the map on the starting location, or fallback to default
if path:
    center_location = coord_dict.get(path[0])
    if center_location:
        center_lat, center_lon = center_location['Latitude'], center_location['Longitude']
    else:
        center_lat, center_lon = 30.3165, 78.0322  # Default: Dehradun
else:
    center_lat, center_lon = 30.3165, 78.0322

# Initialize the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Plot route on map
path_coordinates = []
for stop in path:
    latlon = coord_dict.get(stop)
    if latlon:
        path_coordinates.append((latlon["Latitude"], latlon["Longitude"]))
    else:
        print(f"⚠️ Warning: Coordinates not found for {stop}")

# Draw path line if valid
if path_coordinates:
    folium.PolyLine(
        locations=path_coordinates,
        color="blue", weight=4, opacity=0.6
    ).add_to(m)

# Add markers
for stop in path:
    latlon = coord_dict.get(stop)
    if latlon:
        icon_color = (
            "green" if stop == path[0]
            else "red" if stop == path[-1]
            else "blue"
        )
        folium.Marker(
            [latlon["Latitude"], latlon["Longitude"]],
            popup=stop,
            icon=folium.Icon(color=icon_color)
        ).add_to(m)

# Show the map
m


Missing value counts *before* fix:
Unnamed: 0          0
name                0
clock tower         1
darshanlal chowk    1
saharanpur chowk    1
                   ..
raja road           1
bansiwala           1
manduwala           1
prem nagar          1
balliwala chowk     1
Length: 88, dtype: int64
Missing value counts *after* fix:
Unnamed: 0          0
name                0
clock tower         0
darshanlal chowk    0
saharanpur chowk    0
                   ..
raja road           0
bansiwala           0
manduwala           0
prem nagar          0
balliwala chowk     0
Length: 88, dtype: int64
